In [1726]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB
import numpy as np
# import scipy.sparse as sp

# Classes

In [1727]:
class Dia:
    def __init__(self, id: int, nombre: str):
        self.id = id
        self.nombre = nombre

    def __str__(self) -> str:
        return self.nombre
        # return self.id
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.nombre == __value.nombre
 
class Horario:
    def __init__(self, id: int, inicio: str, fin: str):
        self.id = id
        self.inicio = inicio
        self.fin = fin

    def __str__(self) -> str:
        return self.inicio + "-" + self.fin
        # return self.id
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.inicio == __value.inicio and self.fin == __value.fin
    
class BloqueHorario:
    def __init__(self, dia: Dia, horario: Horario) -> None:
        self.dia = dia
        self.horario = horario
        
    def __str__(self) -> str:
        return str(self.dia) + "_" + str(self.horario)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.dia == __value.dia and self.horario == __value.horario
    
    def id(self):
        return (self.dia.id, self.horario.id)
      
# ver disponibilidad
class Profesor:
    def __init__(self, id: int, nombre: str):
        self.id = id
        self.nombre = nombre
        self.no_disponible = []
        self.prioridades = []

    def __str__(self) -> str:
        return self.nombre
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.nombre == __value.nombre
    
    def disponibilidad(self, bloques_horario):
        disp = bloques_horario.copy()
        for d in self.no_disponible:
            disp.pop((d.dia.id, d.horario.id))
        return disp
        


class Materia:
    def __init__(self,
                 id: int,
                 nombre: str,
                 profesor: Profesor=None,
                 carga_horaria: int=None,
                 cantidad_dias: int=3,
                 horas_maximo: int=10,
                 horas_minimo: int=0,
                 anio: int=None,
                 grupo: int=None
                 ) -> None:
        
        self.nombre = nombre
        self.id = id
        self.carga_horaria = carga_horaria  #C_m
        self.profesor = profesor
        self.cantidad_dias = cantidad_dias  #D_m
        self.horas_maximo = horas_maximo    # Hmax_m
        self.horas_minimo = horas_minimo    # Hmin_m
        self.anio = anio
        self.grupo = grupo
        
    def __str__(self) -> str:
        return self.nombre
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.nombre == __value.nombre

class Prioridad:
    def __init__(self, value: int, profesor: Profesor, bloque_horario: BloqueHorario) -> None:
        self.value = value
        self.profesor = profesor
        self.bloque_horario = bloque_horario
    
    def __str__(self) -> str:
        return str(self.profesor) + "_" + str(self.bloque_horario) + ":prioridad=" + str(self.value)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor and self.bloque_horario == __value.bloque_horario
    
    def id(self):
        return (self.profesor.id, self.bloque_horario.id())


In [1728]:
# variables    
class u:
    def __init__(self, materia: Materia, horario: BloqueHorario) -> None:
        self.materia = materia
        self.horario = horario
        self.variable = None

    def __str__(self) -> str:
        return "u_{" + str(self.materia) + "_" + str(self.horario) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.materia == __value.materia and self.horario == __value.horario
    
class v:
    def __init__(self, materia: Materia, dia: Dia) -> None:
        self.materia = materia
        self.dia = dia
        self.variable = None

    def __str__(self) -> str:
        return "v_{" + str(self.materia) + "_" + str(self.dia) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.materia == __value.materia and self.dia == __value.dia
    
class r:
    def __init__(self, profesor: Profesor) -> None:
        self.profesor = profesor
        self.variable = None
    
    def __str__(self) -> str:
        return "r_{" + str(self.profesor) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor   



# Functions

In [1729]:
def update_disp(profesor, bloques_horario, no_disp_index):
    for i in no_disp_index:
        profesor.no_disponible.append(bloques_horario[i])

def update_prioridad(profesor, bloques_horario, array_prioridad):
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        prior = Prioridad(i[1], profesor, bloques_horario[i[0]])
        if not (prior.bloque_horario in profesor.no_disponible or prior in profesor.prioridades):
            profesor.prioridades.append(prior)


def print_timetable(dias, horarios, u_dict):
    print('\t', *[str(d) for d in dias], sep='\t')
    for h in horarios:
        print(h, *[str(search_materia(BloqueHorario(d,h),u_dict)) for d in dias], sep='\t')

def search_materia(b, u_dict):
    for ui in u_dict:
        u_ob = u_dict[ui]
        if u_ob.horario == b and round(u_ob.variable.x) == 1:
            return u_ob.materia
        
def materias_profesor(profesor, materias_total):
    materias = []

    for m in materias_total:
        if m.profesor == profesor and profesor is not None:
            materias.append(m)
            
    return materias
    


# Formulacion

## (1) Constantes / Datos

In [1730]:
dias = [
    Dia(1, "lunes"),
    Dia(2, "martes"),
    Dia(3, "mierc"),
    Dia(4, "jueves"),
    Dia(5, "viernes")
]
dias_ids = []
for d in dias:
    dias_ids.append(d.id)

horarios = [
    Horario(1, "8:00", "8:50"),
    Horario(2, "8:50", "9:40"),
    Horario(3, "9:50", "10:40"),
    Horario(4, "10:40", "11:30"),
    Horario(5, "11:40", "12:30"),
    Horario(6, "12:30", "13:20")
]
horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)


bloques_horario = {}
for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)


profesores = [
    Profesor(0, "a"),
    Profesor(1, "b"),
    Profesor(2, "c"),
    Profesor(3, "d")
]
profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)



In [1731]:
#disponibilidad
update_disp(profesores[0], bloques_horario, [(1,1),(1,2),(1,5),(1,6),(2,1),(2,2),(3,3),(3,4),(3,5),(3,6),(4,1),(4,2),(4,6),(5,1),(5,2)])
update_disp(profesores[1], bloques_horario, [(1,5),(1,6),(2,1),(2,2),(2,5),(2,6),(3,5),(3,6),(4,1),(4,2),(5,1),(5,2)])
update_disp(profesores[2], bloques_horario, [(1,1),(1,2),(3,1),(3,2),(4,1),(4,2),(5,1),(5,2)])
update_disp(profesores[3], bloques_horario, [(1,5),(1,6),(2,3),(2,4),(3,3),(3,4),(4,3),(4,4),(5,5),(5,6)])

# update_disp(profesores[0], bloques_horario, [0,1,4,5,6,7,14,15,16,17,18,19,23,24,25])
# update_disp(profesores[1], bloques_horario, [4,5,6,7,10,11,16,17,18,19,24,25])
# update_disp(profesores[2], bloques_horario, [0,1,12,13,18,19,24,25])
# update_disp(profesores[3], bloques_horario, [4,5,8,9,14,15,20,21,28,29])

# for p in profesores:
#     print(str(p))
#     print([str(d) for d in p.no_disponible])



In [1732]:
#prioridad
update_prioridad(profesores[0], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),1],
                                                  [(2,4),1],
                                                  [(2,5),3],
                                                  [(2,6),3],
                                                  [(3,1),1],
                                                  [(3,2),1],
                                                  [(4,3),2],
                                                  [(4,4),2],
                                                  [(4,5),2],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  [(5,5),3],
                                                  [(5,6),3]
                                                  ])

update_prioridad(profesores[1], bloques_horario, [[(1,1),1],
                                                  [(1,2),1],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,3),3],
                                                  [(2,4),3],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,3),2],
                                                  [(3,4),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),3],
                                                  [(5,4),3],
                                                  [(5,5),1],
                                                  [(5,6),1],
                                                 ])

update_prioridad(profesores[2], bloques_horario, [[(1,3),1],
                                                  [(1,4),1],
                                                  [(1,5),3],
                                                  [(1,6),3],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,3),2],
                                                  [(2,4),2],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,3),3],
                                                  [(3,4),3],
                                                  [(3,5),2],
                                                  [(3,6),2],
                                                  [(4,3),1],
                                                  [(4,4),1],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,3),2],
                                                  [(5,4),2],
                                                  [(5,5),3],
                                                  [(5,6),3],
                                                  ])

update_prioridad(profesores[3], bloques_horario, [[(1,1),3],
                                                  [(1,2),3],
                                                  [(1,3),1],
                                                  [(1,4),1],
                                                  [(2,1),1],
                                                  [(2,2),1],
                                                  [(2,5),1],
                                                  [(2,6),1],
                                                  [(3,1),2],
                                                  [(3,2),2],
                                                  [(3,5),1],
                                                  [(3,6),1],
                                                  [(4,1),3],
                                                  [(4,2),3],
                                                  [(4,5),1],
                                                  [(4,6),1],
                                                  [(5,1),1],
                                                  [(5,2),1],
                                                  [(5,3),1],
                                                  [(5,4),1],
                                                  ])


In [1733]:
materias = [
    Materia(0, "A", profesores[0], carga_horaria=5, cantidad_dias=2, horas_maximo=3),
    Materia(1, "B", profesores[1], carga_horaria=7, cantidad_dias=3, horas_maximo=3, horas_minimo=2),
    Materia(2, "C", profesores[2], carga_horaria=7, cantidad_dias=3, horas_maximo=3, horas_minimo=2),
    Materia(3, "D", profesores[3], carga_horaria=5, cantidad_dias=2, horas_maximo=3)
]
materias_ids = []
for m in materias:
    materias_ids.append(m.id)

## (2) Variables


$$ u_{mb} = [0,1] $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} = [0,1] $$

    m: materia
    d: dia

$$ r_p = \R $$

    p: profesor


In [1734]:
u_dict = {}
for m in materias:        #crear variables "u" solo para horarios disponibles
    # disp = m.profesor.disponibilidad(bloques_horario)
    for b_id in bloques_horario:
        u_dict[(m.id, b_id)] = u(m, bloques_horario[b_id])

# print([str(us) for us in u_dict])

v_dict = {}
for m in materias:
    for d in dias:
        v_dict[(m.id, d.id)] = v(m, d)

r_dict = {}
for p in profesores:
    r_dict[(p.id)] = r(p)


In [1735]:
print(len(u_dict)) # cantidad de variables
print(u_dict[(0,(1,3))])

120
u_{A_lunes_9:50-10:40}


In [1736]:
# Create a new model
model = gp.Model("timetable")


In [1737]:
# Create variables

# u_vars = m.addMVar(shape=len(u_dict), vtype=GRB.BINARY, name="u") # variable matrix

for u_i in u_dict: # crear variables "u" a partir de u_dict
    u_dict[u_i].variable = model.addVar(vtype=GRB.BINARY, name=str(u_dict[u_i]))

# print([ui.variable for ui in u_dict])

for v_i in v_dict: # crear variables "v" a partir de v_dict
    v_dict[v_i].variable = model.addVar(vtype=GRB.BINARY, name=str(v_dict[v_i]))

# v_vars = np.array([v_i.variable for v_i in v_dict])

for r_i in r_dict:
    r_dict[r_i].variable = model.addVar(vtype=GRB.CONTINUOUS, name=str(r_dict[r_i]))


## (3) Restricciones

### (3.1) unica materia por bloque horario
$$ \sum_m{u_{mb}} \leq 1 $$

    para todo b

In [1738]:
model.addConstrs(gp.quicksum(u_dict[m, b].variable for m in materias_ids) <= 1 for b in bloques_horario_ids)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,


### (3.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = c_m $$

    para todo m (c_m: carga horaria)

In [1739]:
model.addConstrs(gp.quicksum(u_dict[m, b].variable for b in bloques_horario_ids) == materias[m].carga_horaria for m in materias_ids)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### (3.3) definicion de "v"

$$ v_{md} = {OR}_h [u_{mdh}] $$
    para todo m, d

In [1740]:
model.addConstrs(v_dict[(m, d)].variable == gp.or_(u_dict[(m, (d, h))].variable for h in horarios_ids)  for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 4): <gurobi.GenConstr *Awaiting Model Upd

### (3.4) particion de horas por materia

#### (3.4.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [1741]:
model.addConstrs(gp.quicksum(v_dict[m, d].variable for d in dias_ids) == materias[m].cantidad_dias for m in materias_ids)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

#### (3.4.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times Hmin_m \leq \sum_h {u_{mdh}} \leq Hmax_m $$

    para todo m, d

In [1742]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 <= materias[m].horas_maximo for m in materias_ids for d in dias_ids)

model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 >= materias[m].horas_minimo * v_dict[m,d].variable for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>}

### (3.5) horas consecutivas dentro de un dia

$$ u_{mdh} + \sum_h {u_{md(h)} \otimes u_{md(h+1)}} + u_{md(h_{max})} \leq 2 $$

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} \leq 1 $$

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [1743]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:len(horarios_ids)-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)        

{(0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>}

In [1744]:
abc = {1:300, 2:250, 3:200}
abc = {i:i**2 for i in range(5)}
abc = {"a":2, "b":3}
abc = {("a","a"):1, ("a","b"):3}
abc[("b","b")] = 5
t = ("b","b")
# try:
#     print(abc[t])
# except:
#     pass




### (3.6) indisponibilidad

$$ u_{mb} = 0 $$
    para todo m sin disponibilidad en b

In [1745]:
for m in materias:
    for b in m.profesor.no_disponible:
        model.addConstr(u_dict[m.id, b.id()].variable == 0)


### (3.7) unica materia por profesor para un mismo bloque horario

$$ \sum_{m.prof = p} {u_{mb}} \leq 1 $$

    para todo b, p

In [1746]:
# model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in materias_profesor(profesores[p-1], materias)) <= 1 for b in bloques_horario_ids for p in profesores_ids)

for p in profesores:
    mats = materias_profesor(p, materias)
    
    if len(mats) > 1:
        model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)

### (3.8) definicion de rating r_p

$$ r_p = \frac{\sum_{b, m.prof=p} {a_{mb} · u_{mb}}}{\sum_{m.prof=p}{C_m}} $$
$$ r_p \sum_{m.prof=p}{C_m} = \sum_{b, m.prof=p} {a_{mb} · u_{mb}}  $$

In [1747]:
for p in profesores:
    r_coef = 0
    sum_expr = gp.LinExpr()
    for m in materias_profesor(p, materias):
        r_coef += m.carga_horaria
        for pr in p.prioridades:
            sum_expr += pr.value * u_dict[m.id, pr.bloque_horario.id()].variable

    model.addConstr(r_coef * r_dict[p.id].variable == sum_expr)

## (4) Funcion Objetivo

In [1748]:
# Set objective
obj = gp.QuadExpr()

### (4.1) Prioridad horaria de los docentes

$$ \sum_{m,b} {a_{mb}·u_{mb}} $$
alternativa:
$$ \sum_p {r_p}

In [1749]:

# for ui in u_dict:
#     ai = None
#     u_ob = u_dict[ui]
#     for pr in u_ob.materia.profesor.prioridades:
#         if pr.bloque_horario == u_ob.horario:
#             ai = pr
#             break
#     if ai is not None:
#         obj += ai.value * u_ob.variable


# alternativa: ratings
obj += gp.quicksum(r_dict[p].variable for p in profesores_ids)



### (4.2) Minimizar dias consecutivos con la misma materia

$$ \sum_m {\sum_d {v_{m(d)} · v_{m(d+1)}}} $$

In [1750]:
PESO = 5

for m in materias_ids:
    obj += PESO * gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:len(dias_ids)-1])

### (4.3) Maximizar balance global de felicidad (rating): minimizar varianza de los r_p

$$ \frac{1}{P} \sum_p {({r_p - \frac{1}{P} \sum_{p'} {r_{p'}}})^2} $$

In [1751]:
PESO = 50
m = np.mean([r_dict[ri].variable for ri in r_dict])
d2 = np.mean([(r_dict[ri].variable - m)**2 for ri in r_dict])
obj += PESO*d2

## Resolver

In [1752]:

model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 127 rows, 144 columns and 634 nonzeros
Model fingerprint: 0xd8879ef1
Model has 26 quadratic objective terms
Model has 20 quadratic constraints
Model has 20 general constraints
Variable types: 4 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve added 18 rows and 0 columns
Presolve removed 0 rows and 47 columns
Presolve time: 0.00s
Presolved: 302 rows, 144 columns, 965 nonzeros
Presolved model has 46 quadratic objective terms
Variable types: 4 continuous, 140 integer (140 binary)

Root relaxatio

In [1753]:
# result = np.array([var.x for var in model.getVars()])
# print(result)

# print(model.getVars())

# for ui in u_dict:
#     if ui.materia == Materia("A",Profesor("a")):
#         print(str(ui) + " = " + str(ui.variable.x))

# for vi in v_dict:
#     if vi.materia == Materia("A",Profesor("a")):
#         print(str(vi) + " = " + str(vi.variable.x))

# for v in model.getVars():
#     print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % model.ObjVal)
print('=============================================================')
print_timetable(dias, horarios, u_dict)


Obj: 12.3531
		lunes	martes	mierc	jueves	viernes
8:00-8:50	B	C	D	None	D
8:50-9:40	B	C	D	None	D
9:50-10:40	None	A	B	A	D
10:40-11:30	C	A	B	A	B
11:40-12:30	C	A	None	C	B
12:30-13:20	C	None	None	C	B


In [1754]:
for u_i in u_dict:
    u = u_dict[u_i]
    print(str(u) + " = ", u.variable.X)

for v_i in v_dict:
    v = v_dict[v_i]
    print(str(v) + " = ", v.variable.X)

for r_i in r_dict:
    r = r_dict[r_i]
    print(str(r) + " = ", r.variable.X)

u_{A_lunes_8:00-8:50} =  0.0
u_{A_lunes_8:50-9:40} =  0.0
u_{A_lunes_9:50-10:40} =  -0.0
u_{A_lunes_10:40-11:30} =  -0.0
u_{A_lunes_11:40-12:30} =  0.0
u_{A_lunes_12:30-13:20} =  0.0
u_{A_martes_8:00-8:50} =  0.0
u_{A_martes_8:50-9:40} =  0.0
u_{A_martes_9:50-10:40} =  1.0
u_{A_martes_10:40-11:30} =  1.0
u_{A_martes_11:40-12:30} =  1.0
u_{A_martes_12:30-13:20} =  -0.0
u_{A_mierc_8:00-8:50} =  -0.0
u_{A_mierc_8:50-9:40} =  -0.0
u_{A_mierc_9:50-10:40} =  0.0
u_{A_mierc_10:40-11:30} =  0.0
u_{A_mierc_11:40-12:30} =  0.0
u_{A_mierc_12:30-13:20} =  0.0
u_{A_jueves_8:00-8:50} =  0.0
u_{A_jueves_8:50-9:40} =  0.0
u_{A_jueves_9:50-10:40} =  1.0
u_{A_jueves_10:40-11:30} =  1.0
u_{A_jueves_11:40-12:30} =  0.0
u_{A_jueves_12:30-13:20} =  0.0
u_{A_viernes_8:00-8:50} =  0.0
u_{A_viernes_8:50-9:40} =  0.0
u_{A_viernes_9:50-10:40} =  -0.0
u_{A_viernes_10:40-11:30} =  -0.0
u_{A_viernes_11:40-12:30} =  -0.0
u_{A_viernes_12:30-13:20} =  -0.0
u_{B_lunes_8:00-8:50} =  1.0
u_{B_lunes_8:50-9:40} =  1.0
u_{B

In [1755]:
model.write('timetable_opt.lp')

# model.display()